# 개요

## 학습 목표
    - 시간에 따른 특징을 갖는 시계열 데이터 다루기
    - RNN을 이용 주식 데이터 학습 및 다음날 종가 예측

## 학습 순서

<img src='./figs/06_01.png'>

# 이해하기 : RNN
- 순서가 있는 데이터는, 순서를 고려한 모델 설계를 해야 한다
- 모델이 과거와 현재 정보를 다 가지고 있어야 한다.
- 모델이 보낸 과거의 출력을, 다시 입력으로 넣어 주는, 되풀이(recurrent) 되는 구조
- 이를 RNN (recurrent neural network) 라고 한다

<img src='./figs/06_02.png'>

- RNN에서 각 신경망 각 층은 한 시점(cell)
- 모든 시점에서의 가중치는 전부 공유 되고 있음
- 모든 시점에서의 출력은 다음 시점의 입력값과 합쳐져 가중치가 적용
- 최종 출력값이 출력, 출력층 이전의 출력은 은닉 상태
- 입력층은 모델의 시작으로 이전의 은닉 상태가 존재하지 않아 랜덤한 값을 줌

## 장점
- 이전 정보를 현재 시점에 이용, 시간에 대한 정보 추출 가능
- 같은 가중치를 반복 사용, 가중치 수가 적다

## 단점
- 같은 가중치를 여러번 반복 사용, 계산 시간 오래 걸림
- 시계열이 길어질 수록 앞의 정보가 점점 흐려진다

# 데이터 살펴보기
- 넷플릭스 주가의 하루 시가, 고가, 저가 정보로 그날의 종가 예측

In [8]:
import pandas as pd

df = pd.read_csv('./datasets/CH06.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1009 non-null   object 
 1   Open    1009 non-null   float64
 2   High    1009 non-null   float64
 3   Low     1009 non-null   float64
 4   Volume  1009 non-null   int64  
 5   Close   1009 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 47.4+ KB


In [7]:
df.to_csv('./datasets/CH06.csv', index=False)

# 학습용 데이터 만들기

<img src='./figs/06_03.png'>

<img src='./figs/06_04.png'>

# RNN 모델 정의하기

<img src='./figs/06_05.png'>

<img src='./figs/06_06.png'>

<img src='./figs/06_07.png'>

# 모델 학습하기

<img src='./figs/06_08.png'>

# 모델 성능 평가

# 마무리

<img src='./figs/06_09.png'>

# 연습문제

<img src='./figs/06_10.png'>